**分析**

导入库并定义使用 matplotlib 绘制数据的函数。 根据数据，并非所有绘图都会被绘制。

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow import keras
from scipy import io as spio
from keras.layers import Dense
import matplotlib.image as mpimg
from keras.layers import Dropout
from keras.models import Sequential
from sklearn.utils import class_weight
from IPython.display import Image, display

from keras.preprocessing.image import ImageDataGenerator

当前版本的数据集有 1 个 csv 文件：

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[0:10]:
        print(os.path.join(dirname, filename))

In [ ]:
images, dimentions = os.listdir('../input/mpii-dataset-for-python/dataset/images'), 1
plt.subplots(figsize=(35,25))
number=0
for i in range(3):
    for image in images:
        if dimentions == 15:
            break
        number += 1
        cur_img = mpimg.imread("../input/mpii-dataset-for-python/dataset/images/"+image)
        ax = plt.subplot(1, 15, dimentions)
        ax.set_xticks([]), ax.set_yticks([])
        plt.imshow(cur_img)
        dimentions += 1

准备好读入数据并使用绘图函数来可视化数据。

检查文件/kaggle/input/dataset/mpii_dataset.csv

In [ ]:
annots = spio.loadmat('../input/matfile/mpii_human_pose_v1_u12_1.mat')

DF = pd.DataFrame(index=range(0,len(annots['RELEASE'][0][0][0][0])),columns=['Image','Category','Activity'])
DF.dataframeName = 'mpii_human_pose_v1_u12_1.mat'

for i in range(0,len(annots['RELEASE'][0][0][0][0])):
    DF.loc[[i,0],'Image'] = annots["RELEASE"]["annolist"][0,0][0][i]['image']['name'][0, 0][0]
    try: DF.loc[[i,1],'Category'] = annots["RELEASE"]["act"][0,0][:,0][i]["cat_name"][0]
    except Exception: pass
    try: DF.loc[[i,2],'Activity'] = annots["RELEASE"]["act"][0,0][:,0][i]["act_name"][0]
    except Exception: pass
    DF.loc[[i,3],'Train'] = annots["RELEASE"]["img_train"][0,0][0][i]
DF.head()


定义了绘制数据的函数。

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
# 列数据的分布图（直方图/条形图）
frequencies = DF['Category'].value_counts().sort_index().values
freq_series = pd.Series(frequencies)
y_labels = DF['Category'].value_counts().sort_index().index
plt.figure(1, figsize = (15,8))
ax = freq_series.plot(kind='barh', color=(0.2, 0.4, 0.6, 0.6), width=0.9)
ax.set_title('Categories Distribution in Data')
ax.set_xlabel('Images per Class')
ax.set_ylabel('Classes')
ax.set_yticklabels(y_labels)
rects = ax.patches

for rect in rects:
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2
    space = 5
    ha = 'left'
    label = "{:.4f}".format(x_value/18000)
    plt.annotate(label, (x_value, y_value), xytext=(space, 0), textcoords="offset points", va='center', ha=ha) 

In [ ]:
# Correlation matrix
# 相关矩阵
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

    
# plotCorrelationMatrix(DF, 8)

In [ ]:
Cat_DF = pd.DataFrame(DF['Category'].value_counts(), columns=['index', 'Category']).sort_index()
Act_DF = pd.DataFrame(DF['Activity'].value_counts(), columns=['index', 'Activity']).sort_index()

plt.figure(1, figsize = (18,8)) 
plt.subplot(221)  
plt.bar(Cat_DF.index, Cat_DF.Category, color=(0.2, 0.4, 0.6, 0.6))
plt.xticks([])
plt.title('Categories Distribution in Data')  
plt.xlabel(str(len(DF['Category'].value_counts())) + ' Categories Found in Data')
plt.ylabel('Images per Class')
    
plt.subplot(222)  
plt.bar(Act_DF.index, Act_DF.Activity, color=(0.2, 0.4, 0.6, 0.6))
plt.xticks([])
plt.title('Activities Distribution in Data')  
plt.xlabel(str(len(DF['Activity'].value_counts())) + ' Activities Found in Data')
plt.ylabel('Images per Class')

plt.show()

# MODEL VGG16



构建训练集、验证集图像数据生成器

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import ModelCheckpoint

# Train_DF, Test_DF = DF[DF['Train'] == 1].dropna(), DF[DF['Train'] == 0]
# Train_DF = Train_DF.sample(frac=1).reset_index(drop=True)

# datagen = ImageDataGenerator(rescale=1./255,
#                              validation_split=0.2)

# Training   = datagen.flow_from_dataframe(dataframe = Train_DF, 
#                                          directory="../input/mpii-dataset-for-python/dataset/images", 
#                                          x_col="Image", 
#                                          y_col="Category", 
#                                          subset="training",
#                                          batch_size=50, 
#                                          class_mode="sparse", 
#                                          target_size=(224,224))

# Validation = datagen.flow_from_dataframe(dataframe = Train_DF, 
#                                          directory="../input/mpii-dataset-for-python/dataset/images", 
#                                          x_col="Image", 
#                                          y_col="Category", 
#                                          subset="validation", 
#                                          batch_size=50, 
#                                          class_mode="sparse", 
#                                          target_size=(224,224))

A1 [10 Epochs][50 Batch][Shuffling][-10][+layers] - Training & Validation: [93][73]

In [ ]:
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.optimizers import Adam

# vgg16 = VGG16(include_top = False, weights='imagenet', pooling='avg', classes=20)
# model_vgg = Sequential()
# model_vgg.add(vgg16)
# model_vgg.add(keras.layers.Flatten())
# model_vgg.add(Dense(512, activation='relu'))
# model_vgg.add(Dropout(0.5))
# model_vgg.add(Dense(20, activation = 'softmax'))

# for layer in vgg16.layers[:-10]:
#         layer.trainable = False

# adam = Adam(lr=0.0001)
# model_vgg.compile(optimizer = adam, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
# cb_checkpointer_vgg = ModelCheckpoint(filepath = 'best_vgg.h5', monitor = 'val_loss', save_best_only = True)
# fit_history = model_vgg.fit(Training, epochs = 10, validation_data=Validation, callbacks=[cb_checkpointer_vgg])

In [ ]:
# plt.figure(1, figsize = (15,8)) 
      
# plt.subplot(221)  
# plt.plot(fit_history.history['accuracy'])  
# plt.plot(fit_history.history['val_accuracy'])  
# plt.title('model accuracy')  
# plt.ylabel('accuracy')  
# plt.xlabel('epoch')  
# plt.legend(['train', 'val']) 
    
# plt.subplot(222)  
# plt.plot(fit_history.history['loss'])  
# plt.plot(fit_history.history['val_loss'])  
# plt.title('model loss')  
# plt.ylabel('loss')  
# plt.xlabel('epoch')  
# plt.legend(['train', 'val']) 

# plt.show()

# MODEL Xception


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Train_DF, Test_DF = DF[DF['Train'] == 1].dropna(), DF[DF['Train'] == 0]
Train_DF = Train_DF.sample(frac=1).reset_index(drop=True)

datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.10)


Training   = datagen.flow_from_dataframe(dataframe = Train_DF,
                                         directory="../input/mpii-dataset-for-python/dataset/images",
                                         x_col="Image",
                                         y_col="Category",
                                         subset="training",
                                         batch_size=100,
                                         class_mode="sparse",
                                         target_size=(299,299))


Validation = datagen.flow_from_dataframe(dataframe = Train_DF,
                                         directory="../input/mpii-dataset-for-python/dataset/images",
                                         x_col="Image",
                                         y_col="Category",
                                         subset="validation",
                                         batch_size=100,
                                         class_mode="sparse",
                                         target_size=(299,299))


In [ ]:
Xception = keras.applications.xception.Xception(include_top=False, weights='imagenet', pooling='avg', classes=20)
Flatten  = keras.layers.Flatten()
Output  = keras.layers.Dense(20, activation="softmax")

for layer in Xception.layers[:-47]:
    layer.trainable = False

XModelxx = tf.keras.Sequential([ Xception, Flatten, Output])

XModelxx.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

callback = keras.callbacks.EarlyStopping(patience=10,
                                         restore_best_weights=True)

history = XModelxx.fit(Training,
                       epochs=30,
                       callbacks=[callback],
                       validation_data=Validation)

XModelxx.save("XModelxx.h5")

In [ ]:
print(history.history)

In [ ]:
plt.figure(1, figsize = (15,8)) 
      
plt.subplot(221)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'val']) 
    
plt.subplot(222)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'val']) 

plt.show()